# Demo: E$^2$M algorithm for sparse tensor

In [1]:
import sys
sys.path.append("../src/")
import eemix_sparse as eemix_sparse
import sp_tensor
import numpy as np

In [2]:
eemix_sparse.eemix_sparse?

Signature:
eemix_sparse.eemix_sparse(
    T,
    Rs,
    alpha=1.0,
    model=[1, 1, 1, 1],
    max_iter=10,
    iter_inside=1,
    update_rule=0,
    tol=0.0001,
    init_weights=None,
    learn_weights=True,
    init_cp='random',
    init_tucker='random',
    init_train='random',
    verbose=True,
    verbose_interval=1,
    loss_history=True,
    conv_check_interval=10,
    check_sum=False,
    avoid_nan=True,
)
Docstring:
Args:
    T (sp_tensor): sparse tensor
    Rs (list): Ranks Rs = [Rcp (int), Rtucker(list), Rtrain(list)]
    alpha (real number): alpha of alpha-divergence. 
        If alpha = 1.0, then KL div.
        If alpha = 0.5, then Hellinger distance.
    iter_inside(int>0): the number of loop in inside EM-algorithm.
    learn_weights(Boolen): True for learn mixture ratio otherwise False
    init_weights: inital values of weights (eta)
    init_cp: inital values of cp-factors A
    init_tucker: inital values of tucker core G and factors A
    init_train: inital values of

## Example with small toy data

We run the tensor factorization on the small and sparse 4x4x4x4 tensor.
We provide a class 'Sp_tensor' for COO formats. 
Set `normalize=True` to treat the tensor as a discrete probability distribution.  

In [3]:
## define toy small sparse 4x4x4x4 tensor
tensor_size = [4,4,4,4]
coords = [ 
    [1,1,1,0], 
    [1,2,1,0], 
    [1,0,1,0], 
    [2,3,0,2], 
    [2,3,3,2], 
    [0,1,2,1], 
    [2,1,2,1], 
    [1,1,2,1], 
    [0,1,3,1], 
    [3,2,1,3] ]
values = np.array([1,1,1,1,1,1,1,1,1,1])

## Set `normalize=True` to treat the tensor as a discrete probability distribution.  
coo_tensor = sp_tensor.Sp_tensor(coords, values, tensor_size, check_empty=True, normalize=True)

# We can see the number of samples as follows:
print("the number of samples:", coo_tensor.nnz)

the number of samples: 10


In [4]:
## -- model setup -- ## 

# define alpha
alpha = 0.5 # if alpha=1.0, the cost function is the KL div.

# define CP rank # you can change the low-rank structure and try the mixture model. Please also refer to the demo for dense tensors.
model = [1,0,0,0] 
Rcp = 150

Run factorization assuming CP structure. 

In [5]:
factors, P, history, _ = eemix_sparse.eemix_sparse(coo_tensor, [Rcp,0,0], alpha=alpha, model=model, max_iter=100, verbose_interval=10);


EM mixture tensor learning for SPARSE data
Included low-rank structures:
CPD        n_params:1801     Rank :150  
Learn weights            :      True.

Total number of params   :      1801
Sample number in data    :        10
Objective function       :       0.5-div.

Iteration   KL Error      α-div          Weights  CP      Tucker  Train   Noise     Elapsed time
Iter:    10 KL: 0.0005696 α :0.0037293 | Weights: 1.00000 0.00000 0.00000 0.00000 | 0.18 sec.
Iter:    20 KL: 0.0000002 α :0.0000764 | Weights: 1.00000 0.00000 0.00000 0.00000 | 0.34 sec.
Iter:    30 KL: 0.0000000 α :0.0000113 | Weights: 1.00000 0.00000 0.00000 0.00000 | 0.49 sec.
Iter:    40 KL: 0.0000000 α :0.0000038 | Weights: 1.00000 0.00000 0.00000 0.00000 | 0.64 sec.
Iter:    50 KL: 0.0000000 α :0.0000017 | Weights: 1.00000 0.00000 0.00000 0.00000 | 0.80 sec.
Iter:    60 KL: 0.0000000 α :0.0000008 | Weights: 1.00000 0.00000 0.00000 0.00000 | 0.95 sec.
Iter:    70 KL: 0.0000000 α :0.0000004 | Weights: 1.00000 0.00000 0.

`factors` provides low-rank factors.
`P` provides the reconstructed low-rank value of the observed sample.
Using `factors`, we can estimate the non-observed sample.

#### Reconstruction

In [7]:
import utils_mix_sparse as ums
non_obserevd_samples = np.array([ [0,0,0,0], [1,2,0,2] ])
ums.get_vals_from_mixture(non_obserevd_samples, factors)

array([0., 0.])

In [8]:
# You can also obtain all samples in the sample space 
# NOTE: If the sample space is huge, the procedure takes time.
reconst_all_dense = ums.mixture_to_dense(factors, model=model);
print("The total sum of the low-rank reconstruction:", np.sum(reconst_all_dense))

No Tucker structure in this mixture
No Train structure in this mixture
No noise parameter in this mixture
The total sum of the low-rank reconstruction: 1.0


## Example with UCI datasets
In the following code, we donwload UCI datasets and apply E2M algorithm.

#### Dataset preparation

In [9]:
## Load the data<from the UCI datasets and convert it into numpy 
## We here import Balance Scale data (id=12).
## You can easily change the dataset by modifying the id. (e.g., id=19 is for CarEvaluation datasets)
## NOTE: our implementation does not support datasets with missing values

from ucimlrepo import fetch_ucirepo
repo = fetch_ucirepo(id=12)
X = repo.data.features
y = repo.data.targets
X = X.join(y)
X_np = X.to_numpy()

In [10]:
## Let us see the downloaded data
X_np

array([[1, 1, 1, 1, 'B'],
       [2, 1, 1, 1, 'R'],
       [3, 1, 1, 1, 'R'],
       ...,
       [3, 5, 5, 5, 'L'],
       [4, 5, 5, 5, 'L'],
       [5, 5, 5, 5, 'B']], shape=(625, 5), dtype=object)

In [11]:
# X_np includes "str". We convert it into natural numbers.
# Get the dictionary to see the category and number correspondence.
atts = {}
tensor_size = []
names_col = X.columns
for d, col in enumerate(names_col):
    nuq = X[col].unique()
    J = len(nuq)
    att = { nuq[j] : j for j in range(J) }
    atts[d] = att
    tensor_size.append(J)
    print(f"In the column {col}, the unique values are {J}")

print("The tensor size of the dataset is ", tensor_size)

# Define tensor dim
D = len(tensor_size)
# Define non-zero values in the tensor
N = len(X)

# To make npy file in COO format, prepare integer matrix
X_np_coords = np.zeros((N, D), dtype='int64')
for n in range(N):
    categories = X_np[n,:]
    for d in range(D):
        integer = atts[d][categories[d]]
        X_np_coords[n, d] = int(integer)


In the column right-distance, the unique values are 5
In the column right-weight, the unique values are 5
In the column left-distance, the unique values are 5
In the column left-weight, the unique values are 5
In the column class, the unique values are 3
The tensor size of the dataset is  [5, 5, 5, 5, 3]


In [12]:
# Finally, we obtain COO formats
coords, values = np.unique(X_np_coords, axis=0, return_counts=True)
coo_tensor = sp_tensor.Sp_tensor(coords, values, tensor_size, check_empty=True, normalize=True)

#### Model setup and run

In [13]:
# Assuming CP-structure.
Rcp = 50;
model = [1,0,0,0];
# Define the alpha value
alpha = 0.8;

In [14]:
# In the following, we obtain the estimated density behind the data X_np
factors, P, history, _ = eemix_sparse.eemix_sparse(coo_tensor, [Rcp,0,0], alpha=alpha, model=model, max_iter=500, verbose_interval=25);


EM mixture tensor learning for SPARSE data
Included low-rank structures:
CPD        n_params:901      Rank :50   
Learn weights            :      True.

Total number of params   :       901
Sample number in data    :       625
Objective function       :       0.8-div.

Iteration   KL Error      α-div          Weights  CP      Tucker  Train   Noise     Elapsed time
Iter:    25 KL: 0.1486176 α :0.3085234 | Weights: 1.00000 0.00000 0.00000 0.00000 | 5.14 sec.
Iter:    50 KL: 0.1150604 α :0.2201535 | Weights: 1.00000 0.00000 0.00000 0.00000 | 10.11 sec.
Iter:    75 KL: 0.1068327 α :0.1999908 | Weights: 1.00000 0.00000 0.00000 0.00000 | 15.09 sec.
Iter:   100 KL: 0.1029124 α :0.1922534 | Weights: 1.00000 0.00000 0.00000 0.00000 | 20.06 sec.
Iter:   125 KL: 0.1006806 α :0.1885179 | Weights: 1.00000 0.00000 0.00000 0.00000 | 25.02 sec.
Iter:   150 KL: 0.1002619 α :0.1877650 | Weights: 1.00000 0.00000 0.00000 0.00000 | 30.00 sec.
Iter:   175 KL: 0.1000212 α :0.1874051 | Weights: 1.00000 0.000

In [15]:
# If you want to obtain the estimated probability on non-observed samples, then run as follows:
non_obserevd_samples = np.array([ [0,0,0,0,0], [1,2,0,2,2] ])
ums.get_vals_from_mixture(non_obserevd_samples, factors)

array([9.07290721e-04, 7.93880264e-88])